# Encoder-Decoder Model

Here, we will experiment with an Encoder-Decoder model for this sequence-to-sequence problem. We will primarily be following [this tutorial](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html).

Outline:\
I. Imports and Data\
II. Model Creation\
III. Model Training\
IV. Model Evaluation

## I. Imports and Data

In [13]:
import numpy as np
import tensorflow as tf
import pandas as pd
import h5py
from matplotlib import pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, LSTM, Dense, Masking, Normalization, BatchNormalization

from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

In [ ]:
# Load data from the HDF5 file
with h5py.File('enc_dec_preprocessed_data.h5', 'r') as hf:
    x = hf['X'][:]
    y = hf['y'][:]

In [10]:
x[0].shape

(21, 35)

In [11]:
y[0].shape

(35,)

In [42]:
x_train, x_test = x[:-64000], x[-64000:]
y_train, y_test = y[:-64000], y[-64000:]

# Create Decoder Input Data

## Model Creation

In [47]:
norm_layer = Normalization()
norm_layer.adapt(x_train)

### Encoder

In [54]:
NUM_PLAYS = 21
NUM_FEATURES = 35
hidden_size = 128
EPOCHS = 5
BATCH_SIZE = 128


encoder_inputs = Input((NUM_PLAYS, NUM_FEATURES))
# encoder_inputs = Masking(mask_value=-1.1)(encoder_inputs)
# encoder_inputs = norm_layer(encoder_inputs)
encoder = LSTM(hidden_size, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

### Decoder

In [55]:
decoder_inputs = Input((NUM_PLAYS, NUM_FEATURES))
# decoder_inputs = Masking(mask_value=-1.1)(decoder_inputs)
# decoder_inputs = norm_layer(decoder_inputs)
decoder_lstm = LSTM(hidden_size, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(NUM_FEATURES, activation="relu")
decoder_outputs = decoder_dense(decoder_outputs)

### Full Model

In [56]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [57]:
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["accuracy", "MSE"])

In [58]:
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, 21, 35)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_15      │ (None, 21, 35)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_14 (LSTM)      │ [(None, 128),     │     83,968 │ input_layer_14[0… │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_15 (LSTM)      │ [(None, 21, 128), │     83,968 │ input_layer_15[0… │
│                     │ (None, 128),      │            │ lstm_14[0][1],    │
│                     │ (None, 128)]      │            │ lstm_14[0][2]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 21, 35)    │      4,515 │ lstm_15[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 172,451 (673.64 KB)

 Trainable params: 172,451 (673.64 KB)

 Non-trainable params: 0 (0.00 B)

## Model Training

In [59]:
history = model.fit(x=x, y=y, epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/5


ValueError: Layer "functional_6" expects 2 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor 'data:0' shape=(None, 21, 35) dtype=float32>]

# Model Evaluation

# Plot Model

In [ ]:
tf.keras.utils.plot_model(model=model,
         show_shapes=True, to_file='./enc-dec.png')